In [1]:
import import_ipynb
from load_data import train_data
from load_data import song_meta
from load_data import genre_gn_all

importing Jupyter notebook from load_data.ipynb
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115071 entries, 0 to 115070
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   tags          115071 non-null  object
 1   id            115071 non-null  int64 
 2   plylst_title  115071 non-null  object
 3   songs         115071 non-null  object
 4   like_cnt      115071 non-null  int64 
 5   updt_date     115071 non-null  object
dtypes: int64(2), object(4)
memory usage: 5.3+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 707989 entries, 0 to 707988
Data columns (total 9 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   song_gn_dtl_gnr_basket  707989 non-null  object
 1   issue_date              707989 non-null  int64 
 2   album_name              707985 non-null  object
 3   album_id                707989 non-null  int64 
 4   artist_id_baske

In [5]:
import pandas as pd
import seaborn as sns

In [3]:
# playlist_index == 사용할 플레이리스트들의 인덱스를 저장
playlist_index = []
for index in train_data.index:
    tag = train_data['plylst_title'][index]
    if('크리스마스' in tag or '캐럴' in tag or '캐롤' in tag):
        playlist_index.append(index)

In [9]:
#playlist == 사용할 플레이리스트 (train set)
i=0
playlist = pd.DataFrame([], columns=["Index", "tags", "id", "plylst_title", "songs", "like_cnt", "updt_date"])
for row in train_data.itertuples():
    if row.Index in playlist_index :
        if len(row.songs) > 5 :
            playlist.loc[i] = row
            i+= 1

In [14]:
# carol_index == 사용할 노래(캐럴)들의 인덱스 저장
carol_index = []
for index in playlist_index:  # 크리스마스 플레이리스트 번호 돌기
    for song in train_data['songs'][index]:
        if song not in carol_index:
            carol_index.append(song)

In [15]:
#carol_data == 사용할 노래(캐럴) 정보
i=0
carol_data = pd.DataFrame([], columns=["Index","song_gn_dtl_gnr_basket","issue_date","album_name","album_id","artist_id_basket","song_name","song_gn_gnr_basket","artist_name_basket", "id"])
for row in song_meta.itertuples():
    if row.Index in carol_index :
        carol_data.loc[i] = row
        i += 1
carol_data

,Index,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
0,23,"[GN1601, GN1615, GN1610]",20171006,Christmas at Westminster Abbey,10104201,"[345328, 36652, 347399]",Anonymous: Up! Awake! From Highest Steeple - A...,[GN1600],"[Choir Of Westminster Abbey, Simon Preston, Ch...",23
1,41,"[GN1701, GN1703, GN1702]",20121109,Christmas (Deluxe Special Edition),2167838,[35370],Mis Deseos / Feliz Navidad (with Thalia),[GN1700],[Michael Buble],41
2,54,[GN0101],20190408,있어줄래,10271322,[500421],있어줄래,[GN0100],[길구봉구],54
3,70,"[GN0105, GN0101]",20121204,WINTER POETRY,2169041,[1741],너에게 전하지 못한 말,[GN0100],[신혜성],70
4,71,"[GN1304, GN1301]",20110810,Simple Steps,2058451,[614091],Rollercoaster,[GN1300],[Sam Ock],71
...,...,...,...,...,...,...,...,...,...,...
15045,707796,[GN1701],20111207,Christmas Jazz Music,2233676,[757383],O Christmas Tree (Christmas Songs For Family),[GN1700],[Christmas Jazz],707796
15046,707821,[GN0901],20110110,A Kylie Christmas,159204,[2160],Santa Baby,[GN0900],[Kylie Minogue],707821
15047,707913,[GN0101],20131031,Always be there,2213566,[107728],Always be there,[GN0100],[브라운 아이드 소울],707913
15048,707969,"[GN0401, GN0402]",20110902,I Decide (Piano Remix),2010118,[202407],I Decide (Piano Remix),[GN0400],[헤리티지(Heritage)],707969


In [20]:
del playlist['Index'];
del carol_data['Index'];

In [51]:
# 전체 데이터셋의 20 퍼센트를 test_set 으로, 나머지의 20 퍼센트를 validation_set으로
# 나머지를 train_set 으로
from util import write_json
from sklearn.model_selection import train_test_split
train, test = train_test_split(playlist, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)

In [52]:
# 새 데이터들을 processedData 파일에 json 형식으로 저장
write_json(train.to_json(orient = 'records'), "./processedData/playlist_train.json")
write_json(val.to_json(orient = 'records'), "./processedData/playlist_val.json")
write_json(test.to_json(orient = 'records'), "./processedData/playlist_test.json")
write_json(carol_data.to_json(orient = 'records'), "./processedData/carol_data.json")
